In [ ]:
# fit_names = [f"NNPDF40_nnlo_as_0{n}_corr_s{l}" for l in "12" for n in [1140,1150,1160,1170,1175,1180,1185,1190,1195,1200,1210,1220]]

fit_names = [f"220211-ern-00{n}{l}" for l in "ab" for n in range(1,10)]

fit_names = ["230131-rs-nnpdf40-corr-01140-a", "230131-rs-nnpdf40-corr-01140-b", "230131-rs-nnpdf40-corr-01150-a", "230131-rs-nnpdf40-corr-01150-b", "230131-rs-nnpdf40-corr-01160-a", "230131-rs-nnpdf40-corr-01160-b", "230131-rs-nnpdf40-corr-01170-a", "230131-rs-nnpdf40-corr-01170-b", "230131-rs-nnpdf40-corr-01175-a", "230131-rs-nnpdf40-corr-01175-b", "230131-rs-nnpdf40-corr-01180-a", "230131-rs-nnpdf40-corr-01180-b", "230131-rs-nnpdf40-corr-01185-a", "230131-rs-nnpdf40-corr-01185-b", "230131-rs-nnpdf40-corr-01190-a", "230131-rs-nnpdf40-corr-01190-b", "230131-rs-nnpdf40-corr-01195-a", "230131-rs-nnpdf40-corr-01195-b", "230131-rs-nnpdf40-corr-01200-a", "230131-rs-nnpdf40-corr-01200-b", "230131-rs-nnpdf40-corr-01210-a", "230131-rs-nnpdf40-corr-01210-b", "230131-rs-nnpdf40-corr-01220-a", "230131-rs-nnpdf40-corr-01220-b", ]

In [ ]:
from validphys.loader import FallbackLoader as Loader

In [ ]:
from validphys.api import API

In [ ]:
l = Loader()

In [ ]:
fits = [l.check_fit(f) for f in fit_names]

In [ ]:
def measure(replica_data):
    return replica_data.training*3440 + replica_data.validation*1176

In [ ]:
from collections import defaultdict
as_fits = defaultdict(list)
for f in fits:
    th = f.as_input()["theory"]["theoryid"]
    alpha = float(API.theory_info_table(theory_db_id = th).loc["alphas"])
    as_fits[alpha].append(f)
as_fits = dict(as_fits)
    


In [ ]:
indexes = {f: API.fitted_replica_indexes(pdf=f.name) for f in fits}

In [ ]:
replica_data = {f: API.replica_data(fit=f.name) for f in fits}

In [ ]:
import pandas as pd
import numpy as np

In [ ]:
min_values = {}

for alpha, flist in as_fits.items():
    series = []
    for f in flist:
        s = [measure(d) for d in replica_data[f]]
        series.append(pd.Series(s, index=indexes[f]))
    min_values[alpha] = pd.DataFrame(series).min()
    
data = pd.DataFrame(min_values)
    

In [ ]:
mins = {}
for ind, row in data.iterrows():
    a, b, c = np.polyfit(data.columns, row, 2)
    if not np.isnan(b):
        mins[ind] = -b / 2 / a
    
mins = pd.Series(mins)

In [ ]:
f" {mins.mean():.5f} ± {mins.std():.5f} "

In [ ]:
mins.describe()

In [ ]:
from validphys.plotutils import kde_plot

In [ ]:
kde_plot(mins)

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
plt.plot(data.columns, np.array(data.T))